In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gradio torch pandas scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# C2P
import math
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import gradio as gr
from torch.cuda.amp import GradScaler, autocast

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, nhead=8,
                 num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.pos_decoder = PositionalEncoding(d_model, dropout)
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers,
                                          dim_feedforward, dropout, batch_first=True)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask==0, float('-inf')).masked_fill(mask==1, float(0.0))
        return mask

    def forward(self, src, tgt):
        src_emb = self.src_embedding(src) * math.sqrt(self.d_model)
        src_emb = self.pos_encoder(src_emb)
        tgt_emb = self.tgt_embedding(tgt) * math.sqrt(self.d_model)
        tgt_emb = self.pos_decoder(tgt_emb)
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(src.device)
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)
        return self.fc_out(output)

In [5]:
PAD_TOKEN = "<pad>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"

def simple_tokenizer(text):
    return text.strip().split()

def build_vocab(sentences, min_freq=1):
    counts = Counter(token for sentence in sentences for token in sentence)
    vocab = {PAD_TOKEN: 0, SOS_TOKEN: 1, EOS_TOKEN: 2}
    idx = len(vocab)
    for token, count in counts.items():
        if count >= min_freq and token not in vocab:
            vocab[token] = idx
            idx += 1
    return vocab

def numericalize(sentence, vocab):
    return [vocab[SOS_TOKEN]] + [vocab.get(token, vocab[PAD_TOKEN]) for token in sentence] + [vocab[EOS_TOKEN]]

class PrepareDataset(Dataset):
    def __init__(self, data, src_vocab=None, tgt_vocab=None, build_vocabs=False):
        self.df = data.copy()
        self.df["code"] = self.df["code"].fillna("")
        self.df["text"] = self.df["text"].fillna("")
        self.df["src_tokens"] = self.df["code"].apply(simple_tokenizer)
        self.df["tgt_tokens"] = self.df["text"].apply(simple_tokenizer)

        if build_vocabs:
            self.src_vocab = build_vocab(self.df["src_tokens"].tolist())
            self.tgt_vocab = build_vocab(self.df["tgt_tokens"].tolist())
        else:
            self.src_vocab = src_vocab
            self.tgt_vocab = tgt_vocab

        self.df["src_indices"] = self.df["src_tokens"].apply(lambda tokens: numericalize(tokens, self.src_vocab))
        self.df["tgt_indices"] = self.df["tgt_tokens"].apply(lambda tokens: numericalize(tokens, self.tgt_vocab))
        self.data = list(zip(self.df["src_indices"].tolist(), self.df["tgt_indices"].tolist()))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_tensors = [torch.tensor(seq, dtype=torch.long) for seq in src_batch]
    tgt_tensors = [torch.tensor(seq, dtype=torch.long) for seq in tgt_batch]
    src_padded = pad_sequence(src_tensors, batch_first=True, padding_value=0)
    tgt_padded = pad_sequence(tgt_tensors, batch_first=True, padding_value=0)
    return src_padded, tgt_padded

In [7]:
train_df = pd.read_csv("spoc-train.tsv", sep="\t")
eval_df = pd.read_csv("spoc-train-eval.tsv", sep="\t")
test_df = pd.read_csv("spoc-train-test.tsv", sep="\t")

print("Train Data Sample:")
print(train_df[["code", "text"]].head())

Train Data Sample:
                             code  \
0         int gcd(int a, int b) {   
1  return !b ? a : gcd(b, a % b);   
2                               }   
3                    int main() {   
4             int n, nn, ans = 0;   

                                              text  
0                in the function gcd(a,b=integers)  
1  if b=1 return a, else call function gcd(b, a%b)  
2                                              NaN  
3                                              NaN  
4               n , nn, ans = integers with ans =0  


In [8]:
train_dataset = PrepareDataset(train_df, build_vocabs=True)
eval_dataset = PrepareDataset(eval_df, train_dataset.src_vocab, train_dataset.tgt_vocab)
test_dataset = PrepareDataset(test_df, train_dataset.src_vocab, train_dataset.tgt_vocab)

BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
eval_loader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [9]:
def train_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    for src_batch, tgt_batch in tqdm(dataloader, desc="Training"):
        src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
        optimizer.zero_grad()
        tgt_input = tgt_batch[:, :-1]
        tgt_expected = tgt_batch[:, 1:]
        with autocast():
            output = model(src_batch, tgt_input)
            loss = criterion(output.reshape(-1, output.size(-1)), tgt_expected.reshape(-1))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src_batch, tgt_batch in tqdm(dataloader, desc="Evaluating"):
            src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
            tgt_input = tgt_batch[:, :-1]
            tgt_expected = tgt_batch[:, 1:]
            with autocast():
                output = model(src_batch, tgt_input)
                loss = criterion(output.reshape(-1, output.size(-1)), tgt_expected.reshape(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)

In [10]:
model = Transformer(len(train_dataset.src_vocab), len(train_dataset.tgt_vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.src_vocab[PAD_TOKEN])
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = GradScaler()

NUM_EPOCHS = 25
patience = 3
best_eval_loss = float('inf')
patience_counter = 0

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\nEpoch {epoch}")
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, DEVICE)
    eval_loss = evaluate(model, eval_loader, criterion, DEVICE)
    print(f"Train Loss: {train_loss:.4f}, Eval Loss: {eval_loss:.4f}")

    if eval_loss < best_eval_loss:
        best_eval_loss = eval_loss
        torch.save(model.state_dict(), "/content/drive/MyDrive/transformer_code_to_pseudo.pth")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

print("Model saved as '/content/drive/MyDrive/transformer_code_to_pseudo.pth'")

<ipython-input-10-e36ac1c7946f>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Epoch 1


Training:   0%|          | 0/4592 [00:00<?, ?it/s]<ipython-input-9-66f38fc6003c>:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Evaluating:   0%|          | 0/427 [00:00<?, ?it/s]<ipython-input-9-66f38fc6003c>:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Evaluating: 100%|██████████| 427/427 [00:06<00:00, 67.26it/s]


Train Loss: 2.1491, Eval Loss: 1.3711

Epoch 2


Evaluating: 100%|██████████| 427/427 [00:06<00:00, 67.32it/s]


...

Train Loss: 3.3031, Eval Loss: 5.6323

Epoch 24


Evaluating: 100%|██████████| 427/427 [00:06<00:00, 70.22it/s]


Train Loss: nan, Eval Loss: nan

Epoch 25


Evaluating: 100%|██████████| 427/427 [00:06<00:00, 68.11it/s]

Train Loss: nan, Eval Loss: nan
Early stopping triggered.
Model saved as '/content/drive/MyDrive/transformer_code_to_pseudo.pth'


In [11]:
def generate_output(model, src_sentence, src_vocab, tgt_vocab, device, max_len=50):
    model.eval()
    tokens = simple_tokenizer(src_sentence)
    src_indices = numericalize(tokens, src_vocab)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)
    tgt_indices = [tgt_vocab[SOS_TOKEN]]
    with torch.no_grad():
        for _ in range(max_len):
            tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device)
            output = model(src_tensor, tgt_tensor)
            next_token = torch.argmax(output[0, -1, :]).item()
            tgt_indices.append(next_token)
            if next_token == tgt_vocab[EOS_TOKEN]:
                break
    inv_tgt_vocab = {v: k for k, v in tgt_vocab.items()}
    return " ".join([inv_tgt_vocab[idx] for idx in tgt_indices if idx not in (tgt_vocab[SOS_TOKEN], tgt_vocab[EOS_TOKEN])])

In [12]:
model.load_state_dict(torch.load("/content/drive/MyDrive/transformer_code_to_pseudo.pth", map_location=DEVICE))
model.eval()

def generate_pseudocode(code):
    return generate_output(model, code, train_dataset.src_vocab, train_dataset.tgt_vocab, DEVICE)

demo = gr.Interface(
    fn=generate_pseudocode,
    inputs=gr.Textbox(lines=5, placeholder="Enter C++ code here..."),
    outputs=gr.Textbox(label="Generated Pseudocode"),
    title="C++ Code to Pseudocode Converter",
    description="Input C++ code to generate corresponding pseudocode."
)
demo.launch(share=True)

<ipython-input-12-4de2d20211b0>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/transformer_code_to_pseudo.pth", ma

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68f674fa95bfdead25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
